In [0]:
from io import BytesIO
from PIL import Image
from urllib import request
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import numpy as np

In [0]:
STYLED_MAP_BASE_URL = 'http://maps.google.com/maps/api/staticmap?style=element:geometry|color:0x212121&style=element:labels|visibility:off&style=element:labels.icon|visibility:off&style=element:labels.text.fill|color:0x757575&style=element:labels.text.stroke|color:0x212121&style=feature:administrative|element:geometry|color:0x757575|visibility:off&style=feature:administrative.land_parcel|visibility:off&style=feature:administrative.neighborhood|visibility:off&style=feature:landscape|color:0x000000|visibility:simplified&style=feature:landscape|element:geometry.fill|color:0x000000&style=feature:poi|color:0x000000|visibility:off&style=feature:poi|element:labels.text.fill|color:0x757575&style=feature:poi.park|visibility:off&style=feature:poi.park|element:geometry|weight:0.5&style=feature:poi.park|element:labels.text.fill|color:0x616161&style=feature:poi.park|element:labels.text.stroke|color:0x1b1b1b&style=feature:road|color:0xfefcff|weight:0.5&style=feature:road|element:geometry.fill|color:0xfefcff&style=feature:road|element:geometry.stroke|color:0xfefcff&style=feature:road|element:labels.icon|visibility:off&style=feature:road|element:labels.text.fill|color:0x8a8a8a&style=feature:road.arterial|element:geometry|color:0x373737&style=feature:road.arterial|element:geometry.fill|color:0xfefcff&style=feature:road.arterial|element:geometry.stroke|color:0xfefcff&style=feature:road.highway|element:geometry|color:0x3c3c3c&style=feature:road.highway|element:geometry.fill|color:0xfefcff&style=feature:road.highway|element:geometry.stroke|color:0xfefcff&style=feature:road.highway.controlled_access|element:geometry|color:0x4e4e4e&style=feature:road.highway.controlled_access|element:geometry.fill|color:0xfefcff&style=feature:road.highway.controlled_access|element:geometry.stroke|color:0xfefcff&style=feature:road.local|element:geometry.fill|color:0xfefcff&style=feature:road.local|element:geometry.stroke|color:0xfefcff&style=feature:road.local|element:labels.text.fill|color:0x616161&style=feature:transit|color:0x000000|visibility:off&style=feature:transit|element:labels.text.fill|color:0x757575&style=feature:water|color:0x000000&style=feature:water|element:geometry|color:0x000000&style=feature:water|element:labels.text.fill|color:0x3d3d3d'
SATELLITE_MAP_BASE_URL = 'http://maps.googleapis.com/maps/api/staticmap?maptype=satellite'

IMG_W = 512
IMG_H = 512
IMG_D_H = 544 # Deliberately downloading a little bit higher to crop google labels on the bottom of image

ZOOMS = [15, 16]

GOOGLE_MAPS_API_KEY = 'YOUR_API_KEY'

ROOT_FOLDER = 'drive/My Drive/MIPT_MS_THESIS/'

np.random.seed(47)

In [0]:
def download_img_and_mask(lat, lon, idx):
  postfix = "&center={},{}&size={}x{}&zoom={}&sensor=false&key=".format(lat, lon, IMG_W, IMG_D_H, np.random.choice(ZOOMS)) + GOOGLE_MAPS_API_KEY
  
  img_url = SATELLITE_MAP_BASE_URL + postfix
  mask_url = STYLED_MAP_BASE_URL + postfix

  for (url, path) in zip([img_url, mask_url], [ROOT_FOLDER + "styled_road_seg/img", ROOT_FOLDER + "styled_road_seg/mask"]):
    buffer = BytesIO(request.urlopen(url).read())
    image = Image.open(buffer)
    image = image.crop((0, 0, IMG_W, IMG_H))
    image.save(path + "/" + str(idx) + ".png")

In [170]:
junctions = open(ROOT_FOLDER + 'styled_road_seg/junctions.txt', 'r') 
coords = junctions.readlines() 
  
count = 0

for line in tqdm(coords): 
    splitted = re.split(', |\n', line)
    download_img_and_mask(splitted[0], splitted[1], count)
    count += 1

100%|██████████| 101/101 [00:35<00:00,  2.85it/s]
